# Rebalancing Data - XGBoost Model

In [ ]:
# Undersampling VS oversampling
# why not SMOTE ?
# Since SMOTE is KNN based, we are not using SMOTE

random_state =123
X = tree_raw.drop('loan_status',axis=1)
y = tree_raw['loan_status']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=random_state)
X_train,X_validation,y_train,y_validation = train_test_split(X,y,test_size=0.25,random_state=random_state)

cv = KFold(n_splits=5,random_state=random_state,shuffle=True)
collection = []
important_dict ={}

features = list(X_train.columns)

X_train = np.array(X_train)
y_train = np.array(y_train)


for method in ['oversample']:
    one_result = {}
    
    if method == 'class_weight':
        one_result['method'] = method
        print(method)
        rf = RandomForestClassifier(class_weight='balanced')
        model_ap_list = []
        model_f1_list = []
        importance_collection = []
        for fold_train_index,fold_test_index in cv.split(X_train):
            importance_collection = []
            X_train_fold = X_train[fold_train_index]
            X_test_fold = X_train[fold_test_index]
            y_train_fold = y_train[fold_train_index]
            y_test_fold = y_train[fold_test_index]
            rf.fit(X_train_fold,y_train_fold)
            
            fold_predict = rf.predict(X_test_fold)
            fold_pred_prob = rf.predict_proba(X_test_fold)
            model_f1 = f1_score(y_true=y_test_fold,y_pred=fold_predict)
            model_ap = average_precision_score(y_true=y_test_fold,y_score=fold_pred_prob[:,1])
            
            print(model_f1,model_ap)
            model_f1_list.append(model_f1)
            model_ap_list.append(model_ap)
            
            importances = rf.feature_importances_
            importance = pd.Series(importances,index=X.columns)
            df = pd.DataFrame(importance).reset_index()
            
            importance_collection.append(df)
        important_dict[method] = importance_collection 
        one_result['f1'] = np.mean(model_f1)
        one_result['ap'] = np.mean(model_ap)
        collection.append(one_result)
    
    if method == 'oversample':
        one_result['method'] = method
        print(method)
        xg = xgb.XGBClassifier()
        oversampler = RandomOverSampler(sampling_strategy='minority')
        
        model_ap_list = []
        model_f1_list = []
        importance_collection = []
        for fold_train_index,fold_test_index in cv.split(X_train):
            
            X_train_fold = X_train[fold_train_index]
            X_test_fold = X_train[fold_test_index]
            y_train_fold = y_train[fold_train_index]
            y_test_fold = y_train[fold_test_index]
            
            X_train_fold,y_train_fold = oversampler.fit_resample(X_train_fold,y_train_fold)
            xg.fit(X_train_fold,y_train_fold)
            
            fold_predict = xg.predict(X_test_fold)
            fold_pred_prob = xg.predict_proba(X_test_fold)
            model_f1 = f1_score(y_true=y_test_fold,y_pred=fold_predict)
            model_ap = average_precision_score(y_true=y_test_fold,y_score=fold_pred_prob[:,1])
            
            model_f1_list.append(model_f1)
            model_ap_list.append(model_ap)
            print(model_f1,model_ap)
            importances = xg.feature_importances_
            importance = pd.Series(importances,index=X.columns)
           
            
            importance_collection.append(importance)
        important_dict[method] = importance_collection 
        one_result['f1'] = np.mean(model_f1)
        one_result['ap'] = np.mean(model_ap)
        collection.append(one_result)
    if method == 'undersample':
        one_result['method'] = method
        print(method)
        xg = xgb.XGBClassifier()
        undersampler = RandomUnderSampler(sampling_strategy='majority')
        
        model_ap_list = []
        model_f1_list = []
        importance_collection = []
        for fold_train_index,fold_test_index in cv.split(X_train):
            
            X_train_fold = X_train[fold_train_index]
            X_test_fold = X_train[fold_test_index]
            y_train_fold = y_train[fold_train_index]
            y_test_fold = y_train[fold_test_index]
            
            X_train_fold,y_train_fold = oversampler.fit_resample(X_train_fold,y_train_fold)
            xg.fit(X_train_fold,y_train_fold)
            
            fold_predict = xg.predict(X_test_fold)
            fold_pred_prob = xg.predict_proba(X_test_fold)
            
            model_f1 = f1_score(y_true=y_test_fold,y_pred=fold_predict)
            model_ap = average_precision_score(y_true=y_test_fold,y_score=fold_pred_prob[:,1])
            
            model_f1_list.append(model_f1)
            model_ap_list.append(model_ap)
            print(model_f1,model_ap)
            importances = xg.feature_importances_
            importance = pd.Series(importances,index=X.columns)
            importance_collection.append(importance)
        
        important_dict[method] = importance_collection 
        one_result['f1'] = np.mean(model_f1)
        one_result['ap'] = np.mean(model_ap)
        collection.append(one_result)

In [506]:
rebalance_result = pd.DataFrame(collection)
# cross validation result 
rebalance_result
# oversample is the best rebalancing for XGboost

,method,f1,ap
0,oversample,0.529885,0.501456
1,undersample,0.528924,0.499260


For XGBoost, oversampling outperforms